## PREPARATION DES DONNEES POUR ANALYSE ET MODELISATION

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk 
import datetime
import re
import pyproj

### Importation des données

In [2]:
DEPT_CODE = 73  # à changer facilement ici

# --- Chargement des fichiers ---
path_existants = f"../data/donnees_dpe_existants_{DEPT_CODE}.csv"
path_neufs = f"../data/donnees_dpe_neufs_{DEPT_CODE}.csv"

logements_existants = pd.read_csv(path_existants)
logements_neufs = pd.read_csv(path_neufs)

print(f"✅ Chargé DPE {DEPT_CODE} : {logements_existants.shape[0]} existants, {logements_neufs.shape[0]} neufs")

C:\Users\ychen\AppData\Local\Temp\ipykernel_30540\2418907252.py:7: DtypeWarning: Columns (7,26,39,51,201,202,203,204,208,213) have mixed types. Specify dtype option on import or set low_memory=False.
  logements_existants = pd.read_csv(path_existants)


✅ Chargé DPE 73 : 110526 existants, 10816 neufs


C:\Users\ychen\AppData\Local\Temp\ipykernel_30540\2418907252.py:8: DtypeWarning: Columns (31,37,136,137,138,139,140,143,144,145,147,148,149,151,152,153,154,157,158,159,166,169,170,171,175,177,178,179,180,191,199) have mixed types. Specify dtype option on import or set low_memory=False.
  logements_neufs = pd.read_csv(path_neufs)


In [3]:
# Ajouter de la colonne "Logement" dans chaque DF
logements_existants["Logement"]="Ancien"
logements_neufs["Logement"]="Neuf"

In [4]:
# Ajout année de construction
now=datetime.datetime.now()
logements_neufs["annee_construction"]=now.year
print (logements_neufs["annee_construction"].value_counts())

annee_construction
2025    10816
Name: count, dtype: int64


In [5]:
# Fusionner les deux DF
logements_merged=pd.concat([logements_neufs,logements_existants], join = 'inner')
print(logements_merged.columns)
print(len(logements_merged.columns))

Index(['numero_dpe', 'date_derniere_modification_dpe',
       'date_visite_diagnostiqueur', 'date_etablissement_dpe',
       'date_reception_dpe', 'date_fin_validite_dpe', 'numero_dpe_remplace',
       'numero_dpe_immeuble_associe', 'id_rnb', 'provenance_id_rnb',
       ...
       'surface_totale_capteurs_pv', 'nombre_module',
       'production_electricite_pv_kwhep_par_an',
       'electricite_pv_autoconsommee', '_geopoint', '_id', '_i', '_rand',
       'Logement', 'annee_construction'],
      dtype='object', length=209)
209


In [6]:
# Créer une colonne avec uniquement l'année de la Date de réception du DPE
logements_merged["annee_reception_DPE"]=logements_merged["date_reception_dpe"].str[:4]
print(logements_merged[["annee_reception_DPE","date_reception_dpe"]])

       annee_reception_DPE date_reception_dpe
0                     2021         2021-07-01
1                     2021         2021-07-02
2                     2021         2021-07-02
3                     2021         2021-07-08
4                     2021         2021-07-09
...                    ...                ...
110521                2025         2025-10-27
110522                2025         2025-10-27
110523                2025         2025-10-27
110524                2025         2025-10-27
110525                2025         2025-10-27

[121342 rows x 2 columns]


C:\Users\ychen\AppData\Local\Temp\ipykernel_30540\2634798736.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  logements_merged["annee_reception_DPE"]=logements_merged["date_reception_dpe"].str[:4]


In [7]:
# Créer une colonne periode_construction avec ces classes ci-dessous
def periode_construction(année):
    if année < 1960:
        return "Avant 1960"
    elif 1960 <= année <= 1970:
        return "1961 - 1970" 
    elif 1971 <= année <= 1980:
        return "1971 - 1980" 
    elif 1981 <= année <= 1990:
        return "1981 - 1990"
    elif 1991 <= année <= 2000:
        return "1991 - 2000"
    elif 2001 <= année <= 2010:
        return "2001 - 2010"
    elif pd.isna(année):
        return "Année inconnue"
    else:
        return "Après 2010"

logements_merged["periode_construction"]=logements_merged["annee_construction"].apply(periode_construction)
print(logements_merged[["periode_construction","annee_construction"]])

       periode_construction  annee_construction
0                Après 2010              2025.0
1                Après 2010              2025.0
2                Après 2010              2025.0
3                Après 2010              2025.0
4                Après 2010              2025.0
...                     ...                 ...
110521           Après 2010              2015.0
110522           Avant 1960              1930.0
110523       Année inconnue                 NaN
110524           Avant 1960              1900.0
110525       Année inconnue                 NaN

[121342 rows x 2 columns]


C:\Users\ychen\AppData\Local\Temp\ipykernel_30540\1581620481.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  logements_merged["periode_construction"]=logements_merged["annee_construction"].apply(periode_construction)


In [8]:
pd.set_option('display.max_columns', None)  # Affiche toutes les colonnes
logements_merged.head(10)

,numero_dpe,date_derniere_modification_dpe,date_visite_diagnostiqueur,date_etablissement_dpe,date_reception_dpe,date_fin_validite_dpe,numero_dpe_remplace,numero_dpe_immeuble_associe,id_rnb,provenance_id_rnb,numero_rpls_logement,numero_immatriculation_copropriete,modele_dpe,version_dpe,methode_application_dpe,etiquette_dpe,etiquette_ges,classe_altitude,zone_climatique,type_batiment,type_installation_chauffage,type_installation_ecs,hauteur_sous_plafond,nombre_appartement,nombre_niveau_immeuble,nombre_niveau_logement,typologie_logement,appartement_non_visite,surface_habitable_immeuble,surface_habitable_logement,surface_tertiaire_immeuble,classe_inertie_batiment,adresse_ban,numero_voie_ban,nom_rue_ban,nom_commune_ban,code_postal_ban,code_insee_ban,code_departement_ban,code_region_ban,identifiant_ban,coordonnee_cartographique_x_ban,coordonnee_cartographique_y_ban,score_ban,statut_geocodage,adresse_brut,nom_commune_brut,code_postal_brut,numero_etage_appartement,position_logement_dans_immeuble,nom_residence,complement_adresse_batiment,complement_adresse_logement,indicateur_confort_ete,protection_solaire_exterieure,logement_traversant,presence_brasseur_air,inertie_lourde,isolation_toiture,deperditions_enveloppe,deperditions_ponts_thermiques,deperditions_murs,deperditions_planchers_hauts,deperditions_planchers_bas,deperditions_portes,deperditions_baies_vitrees,deperditions_renouvellement_air,qualite_isolation_enveloppe,qualite_isolation_murs,qualite_isolation_plancher_haut_comble_amenage,qualite_isolation_plancher_bas,qualite_isolation_menuiseries,ubat_w_par_m2_k,besoin_chauffage,besoin_ecs,besoin_refroidissement,conso_5_usages_ep,conso_5_usages_par_m2_ep,conso_chauffage_ep,conso_ecs_ep,conso_refroidissement_ep,conso_eclairage_ep,conso_auxiliaires_ep,conso_5_usages_ef,conso_5_usages_par_m2_ef,conso_chauffage_ef,conso_ecs_ef,conso_refroidissement_ef,conso_eclairage_ef,conso_auxiliaires_ef,emission_ges_5_usages,emission_ges_5_usages_par_m2,emission_ges_chauffage,emission_ges_ecs,emission_ges_refroidissement,emission_ges_eclairage,emission_ges_auxiliaires,type_energie_n1,conso_5_usages_ef_energie_n1,conso_chauffage_ef_energie_n1,conso_ecs_ef_energie_n1,cout_total_5_usages_energie_n1,cout_chauffage_energie_n1,cout_ecs_energie_n1,emission_ges_5_usages_energie_n1,emission_ges_chauffage_energie_n1,emission_ges_ecs_energie_n1,type_energie_n2,conso_5_usages_ef_energie_n2,conso_chauffage_ef_energie_n2,conso_ecs_ef_energie_n2,cout_total_5_usages_energie_n2,cout_chauffage_energie_n2,cout_ecs_energie_n2,emission_ges_5_usages_energie_n2,emission_ges_chauffage_energie_n2,emission_ges_ecs_energie_n2,type_energie_n3,conso_5_usages_ef_energie_n3,conso_chauffage_ef_energie_n3,conso_ecs_ef_energie_n3,cout_total_5_usages_energie_n3,cout_chauffage_energie_n3,cout_ecs_energie_n3,emission_ges_5_usages_energie_n3,emission_ges_chauffage_energie_n3,emission_ges_ecs_energie_n3,cout_total_5_usages,cout_chauffage,cout_ecs,cout_refroidissement,cout_eclairage,cout_auxiliaires,type_energie_principale_chauffage,type_generateur_chauffage_principal,type_installation_chauffage_n1,type_emetteur_installation_chauffage_n1,configuration_installation_chauffage_n1,description_installation_chauffage_n1,conso_chauffage_installation_chauffage_n1,surface_chauffee_installation_chauffage_n1,type_generateur_n1_installation_n1,type_energie_generateur_n1_installation_n1,usage_generateur_n1_installation_n1,conso_chauffage_generateur_n1_installation_n1,type_generateur_n2_installation_n1,type_energie_generateur_n2_installation_n1,usage_generateur_n2_installation_n1,conso_chauffage_generateur_n2_installation_n1,type_installation_chauffage_n2,type_emetteur_installation_chauffage_n2,configuration_installation_chauffage_n2,description_installation_chauffage_n2,conso_chauffage_installation_chauffage_n2,surface_chauffee_installation_chauffage_n2,type_generateur_n1_installation_n2,type_energie_generateur_n1_installation_n2,usage_generateur_n1_installation_n2,conso_chauffage_generateur_n1_installa

In [9]:
# ---Filtrage des types de bâtiment ---
print("\nRépartition avant filtrage :")
print(logements_merged["type_batiment"].value_counts(dropna=False))

# On garde uniquement les logements individuels (maisons et appartements)
logements_merged = logements_merged[
    logements_merged["type_batiment"].isin(["maison", "appartement"])
]

print("\n✅ Filtrage effectué.")
print(logements_merged["type_batiment"].value_counts(dropna=False))
print(f"📊 Nouvelle dimension du DF : {logements_merged.shape}")


Répartition avant filtrage :
type_batiment
appartement    96964
maison         20744
immeuble        3634
Name: count, dtype: int64

✅ Filtrage effectué.
type_batiment
appartement    96964
maison         20744
Name: count, dtype: int64
📊 Nouvelle dimension du DF : (117708, 211)


In [10]:
# 🌍 Conversion des coordonnées Lambert93 → WGS84 (latitude / longitude)
# Vérification des colonnes
if {"coordonnee_cartographique_x_ban", "coordonnee_cartographique_y_ban"}.issubset(logements_merged.columns):
    print("Conversion des coordonnées Lambert93 (EPSG:2154) vers WGS84 (EPSG:4326)...")

    # Création du transformateur
    transformer = pyproj.Transformer.from_crs("EPSG:2154", "EPSG:4326", always_xy=True)

    # Transformation vectorisée
    logements_merged["longitude"], logements_merged["latitude"] = transformer.transform(
        logements_merged["coordonnee_cartographique_x_ban"].values,
        logements_merged["coordonnee_cartographique_y_ban"].values
    )

    # Vérification rapide de la transformation
    print("✅ Conversion terminée.")
    print("Exemple de coordonnées converties :")
    print(logements_merged[["coordonnee_cartographique_x_ban", "coordonnee_cartographique_y_ban", "longitude", "latitude"]].head(3))

    # Optionnel : suppression des anciennes colonnes pour alléger le dataset
    # (décommente si tu veux les retirer après conversion)
    logements_merged = logements_merged.drop(columns=["coordonnee_cartographique_x_ban", "coordonnee_cartographique_y_ban"])

else:
    print("⚠️ Les colonnes de coordonnées Lambert93 ne sont pas présentes dans le dataframe.")

Conversion des coordonnées Lambert93 (EPSG:2154) vers WGS84 (EPSG:4326)...
✅ Conversion terminée.
Exemple de coordonnées converties :
   coordonnee_cartographique_x_ban  coordonnee_cartographique_y_ban  \
0                        927396.82                       6518105.11   
1                        946042.17                       6501898.56   
2                        912675.52                       6492089.43   

   longitude   latitude  
0   5.924250  45.724631  
1   6.155453  45.572340  
2   5.723560  45.495255  


C:\Users\ychen\AppData\Local\Temp\ipykernel_30540\484049425.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  logements_merged["longitude"], logements_merged["latitude"] = transformer.transform(
C:\Users\ychen\AppData\Local\Temp\ipykernel_30540\484049425.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  logements_merged["longitude"], logements_merged["latitude"] = transformer.transform(


Gestion des valeurs manquantes

In [11]:
# Identifier et supprimer les colonnes entièrement vides
empty_cols = logements_merged.columns[logements_merged.isna().all()]
logements_merged = logements_merged.drop(columns=empty_cols)

# Supprimer colonnes inutiles
cols_to_drop = ["_geopoint", "_id", "_rand", "_i"]
existing_cols = [col for col in cols_to_drop if col in logements_merged.columns]
logements_merged = logements_merged.drop(columns=existing_cols)

print(f"✅ {len(empty_cols)} colonnes vides supprimées.")
print(f"✅ Colonnes inutiles supprimées : {existing_cols}")
print(f"📊 Nouvelle dimension du DF : {logements_merged.shape}")

✅ 2 colonnes vides supprimées.
✅ Colonnes inutiles supprimées : ['_geopoint', '_id', '_rand', '_i']
📊 Nouvelle dimension du DF : (117708, 205)


In [12]:
# Pourcentage de valeurs manquantes par colonne
missing_ratio = logements_merged.isna().mean().sort_values(ascending=False)
print(missing_ratio.head(100))  # affiche les 20 colonnes les plus vides

type_energie_generateur_n2_ecs_n1          0.999983
type_generateur_n2_ecs_n1                  0.999983
usage_generateur_n2_ecs_n1                 0.999983
description_generateur_n2_ecs_n1           0.999983
volume_stockage_generateur_n2_ecs_n1       0.999983
                                             ...   
volume_stockage_generateur_n1_ecs_n1       0.114903
type_energie_generateur_n1_ecs_n1          0.114631
type_generateur_chauffage_principal_ecs    0.114631
usage_generateur_n1_ecs_n1                 0.114631
configuration_installation_ecs_n1          0.114631
Length: 100, dtype: float64


In [13]:
print("📊 Taux moyen de valeurs manquantes avant nettoyage :")
print(f"{logements_merged.isna().mean().mean():.2%}")

# Sélection des colonnes à supprimer (plus de 80% de valeurs manquantes)
cols_to_drop = missing_ratio[missing_ratio > 0.5].index
print(f"📉 Colonnes à supprimer (>{50}% de valeurs manquantes) : {len(cols_to_drop)}")

# Suppression de ces colonnes
logements_merged = logements_merged.drop(columns=cols_to_drop)
print(f"✅ Colonnes supprimées : {len(cols_to_drop)}")
print(f"📊 Nouvelle dimension du DF : {logements_merged.shape}")

print("📊 Taux moyen de valeurs manquantes après nettoyage :")
print(f"{logements_merged.isna().mean().mean():.2%}")

📊 Taux moyen de valeurs manquantes avant nettoyage :
33.61%
📉 Colonnes à supprimer (>50% de valeurs manquantes) : 62
✅ Colonnes supprimées : 62
📊 Nouvelle dimension du DF : (117708, 143)
📊 Taux moyen de valeurs manquantes après nettoyage :
8.99%


In [14]:
# --- 💾 Sauvegarde du jeu de données nettoyé ---
output_path = f"../data/donnees_dpe_{DEPT_CODE}_clean.csv"
logements_merged.to_csv(output_path, index=False)

print("\n✅ Données nettoyées sauvegardées avec succès.")
print(f"📁 Fichier exporté : {output_path}")

# --- 🔍 Vérification rapide ---
print("\nAperçu final :")
display(logements_merged.head(3))

print("\nTypes de données :")
print(logements_merged.dtypes.head(10))

print("\nTaux de valeurs manquantes par colonne (top 10) :")
print(logements_merged.isna().mean().sort_values(ascending=False).head(10))



✅ Données nettoyées sauvegardées avec succès.
📁 Fichier exporté : ../data/donnees_dpe_73_clean.csv

Aperçu final :


,numero_dpe,date_derniere_modification_dpe,date_visite_diagnostiqueur,date_etablissement_dpe,date_reception_dpe,date_fin_validite_dpe,modele_dpe,version_dpe,methode_application_dpe,etiquette_dpe,etiquette_ges,classe_altitude,zone_climatique,type_batiment,type_installation_chauffage,type_installation_ecs,hauteur_sous_plafond,nombre_appartement,nombre_niveau_logement,surface_habitable_immeuble,surface_habitable_logement,classe_inertie_batiment,adresse_ban,numero_voie_ban,nom_rue_ban,nom_commune_ban,code_postal_ban,code_insee_ban,code_departement_ban,code_region_ban,identifiant_ban,score_ban,statut_geocodage,adresse_brut,nom_commune_brut,code_postal_brut,numero_etage_appartement,complement_adresse_batiment,complement_adresse_logement,indicateur_confort_ete,protection_solaire_exterieure,logement_traversant,presence_brasseur_air,inertie_lourde,isolation_toiture,deperditions_enveloppe,deperditions_ponts_thermiques,deperditions_murs,deperditions_planchers_hauts,deperditions_planchers_bas,deperditions_portes,deperditions_baies_vitrees,deperditions_renouvellement_air,qualite_isolation_enveloppe,qualite_isolation_murs,qualite_isolation_plancher_bas,qualite_isolation_menuiseries,ubat_w_par_m2_k,besoin_chauffage,besoin_ecs,besoin_refroidissement,conso_5_usages_ep,conso_5_usages_par_m2_ep,conso_chauffage_ep,conso_ecs_ep,conso_refroidissement_ep,conso_eclairage_ep,conso_auxiliaires_ep,conso_5_usages_ef,conso_5_usages_par_m2_ef,conso_chauffage_ef,conso_ecs_ef,conso_refroidissement_ef,conso_eclairage_ef,conso_auxiliaires_ef,emission_ges_5_usages,emission_ges_5_usages_par_m2,emission_ges_chauffage,emission_ges_ecs,emission_ges_refroidissement,emission_ges_eclairage,emission_ges_auxiliaires,type_energie_n1,conso_5_usages_ef_energie_n1,conso_chauffage_ef_energie_n1,conso_ecs_ef_energie_n1,cout_total_5_usages_energie_n1,cout_chauffage_energie_n1,cout_ecs_energie_n1,emission_ges_5_usages_energie_n1,emission_ges_chauffage_energie_n1,emission_ges_ecs_energie_n1,type_energie_n2,conso_5_usages_ef_energie_n2,conso_chauffage_ef_energie_n2,conso_ecs_ef_energie_n2,cout_total_5_usages_energie_n2,cout_chauffage_energie_n2,cout_ecs_energie_n2,emission_ges_5_usages_energie_n2,emission_ges_chauffage_energie_n2,emission_ges_ecs_energie_n2,cout_total_5_usages,cout_chauffage,cout_ecs,cout_refroidissement,cout_eclairage,cout_auxiliaires,type_energie_principale_chauffage,type_generateur_chauffage_principal,type_installation_chauffage_n1,type_emetteur_installation_chauffage_n1,configuration_installation_chauffage_n1,description_installation_chauffage_n1,conso_chauffage_installation_chauffage_n1,surface_chauffee_installation_chauffage_n1,type_generateur_n1_installation_n1,type_energie_generateur_n1_installation_n1,usage_generateur_n1_installation_n1,conso_chauffage_generateur_n1_installation_n1,type_energie_principale_ecs,type_generateur_chauffage_principal_ecs,type_installation_ecs_n1,configuration_installation_ecs_n1,description_installation_ecs_n1,conso_ef_installation_ecs_n1,nombre_logements_desservis_par_installation_ecs_n1,surface_habitable_desservie_par_installation_ecs_n1,type_installation_solaire_n1,type_generateur_n1_ecs_n1,type_energie_generateur_n1_ecs_n1,usage_generateur_n1_ecs_n1,description_generateur_n1_ecs_n1,volume_stockage_generateur_n1_ecs_n1,conso_ef_generateur_n1_ecs_n1,ventilation_posterieure_2012,production_electricite_pv_kwhep_par_an,Logement,annee_construction,annee_reception_DPE,periode_construction,longitude,latitude
0,2173N0055368S,2023-11-08,2021-06-30,2021-06-30,2021-07-01,2031-06-29,DPE NEUF logement : RT2012,1.0,dpe issu d'une étude thermique réglementaire R...,A,A,NaN,NaN,maison,NaN,NaN,2.5,1.0,1.0,95.4,95.4,NaN,Rue de la Gare 73100 Grésy-sur-Aix,NaN,271 RUE DE LA GARE,Grésy-sur-Aix,73100,73128,73.0,84.0,73128_0144,0.55,adresse géocodée ban à l'adresse,271 RUE DE LA GARE,Grésy-sur-Aix,73100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,très bonne,très bonne,très bonne,très bonne,0.39,NaN,NaN,NaN,5956.5,62.3,3859


Types de données :
numero_dpe                         object
date_derniere_modification_dpe     object
date_visite_diagnostiqueur         object
date_etablissement_dpe             object
date_reception_dpe                 object
date_fin_validite_dpe              object
modele_dpe                         object
version_dpe                       float64
methode_application_dpe            object
etiquette_dpe                      object
dtype: object

Taux de valeurs manquantes par colonne (top 10) :
inertie_lourde                       0.468150
isolation_toiture                    0.462645
protection_solaire_exterieure        0.459994
logement_traversant                  0.459994
presence_brasseur_air                0.459994
indicateur_confort_ete               0.455950
type_energie_n2                      0.365192
emission_ges_chauffage_energie_n2    0.365192
emission_ges_ecs_energie_n2          0.365192
conso_5_usages_ef_energie_n2         0.365192
dtype: float64
